# Week 5

## 5.1 Continuous Time Collision Detection

- In instantaneous time detection, when the time step is large enough, it can miss the tunneling effect between two colliding bodies.
    - So, we ask: Does there exist a time instant when A and B overlap?
    - When are A and B approaching?
    
- **f(t)** < 0 defines when two balls overlap
- **g(t)** < 0 define when two balls are approaching

- **The interval when f(t) and g(t) both are negative is the interval where they collide**

![](https://i.ibb.co/RNQ3VCS/Screenshot-from-2019-04-09-16-24-49.png)

## 5.3 Overlapping Polynomial

- Now, we need to determine the time intervals when both the polynomials f(t) and g(t) are either +ve or -ve

- We find the roots of both f(t) and g(t), i.e. points where they touch the origin (and see their sign on those points)
- Sorting the timestamps and finding the overlap will enable us to see the overlap of the interval for both the polynomials

![](https://i.ibb.co/SRBZ2MP/Screenshot-from-2019-04-09-16-40-24.png)

## 5.4 Collision Response Pipeline

- When we are using a time integrator (either explicit or implicit euler) to determine the next position and velocity of a body, we need to determine the time interval when the bodies may suffer from the tunneling effect
- In such case, we first determine the ending position and velocity **qe** and **ve** from the starting position and velocity **qs** and **vs**
- Then we determine the overlapping interval
- We then calculate the impulse that needs to be applied at the start of the overlapping interval
    - After the impulse gets applied the final position and velocity of the body get changed
        - We take care of this by providing a feedback to the integrator about the application of the impulse to the body
        
![](https://i.ibb.co/3scp2fW/Screenshot-from-2019-04-09-17-10-00.png)

## 5.5 Adding Impulses to Time Integration Loop

- When we calculate the impulse as mentioned above, we feedback that to the time integrator at the starting timestep i.e at **ts**
    - We do this because in a complex system, if we account the impulse only when the overlapping time interval starts, we might encounter unwanted collisions of the bodies (although it might seem realistic)


![](https://i.ibb.co/vqwCVjT/Screenshot-from-2019-04-09-17-15-13.png)

## 5.6 Iterating over Multiple Simultaneous Collisions

### Steps (the iterative approach, more likely not to converge even when COR=1)

1. We integrate the dynamics of the balls, i.e. we first determine the end positions and velocities
    - Tentative End of State (EoS) positions
2. We then apply the **Continuous Time Collision Detection** method to find the overlap/tunneling time intervals. **(collision times)**
    - We now need to fix/prevent the collision
3. Apply collision impulses at the start of timestep (and not on instantaneous collision time)
    - Now repeat the first two steps again
    - When the timestep does not detect any collision, we are done
    - **NOTE:** Using "Time of Event" approach compels us to look for all the possible collisions in a scene, and its complications grow exponentially as the number of colliding bodies increases. So, the above steps are choosen, where the impulses are applied at the start of a timestep


## 5.8 Parallel vs Sequential Impulses

- When two bodies simultaneously collide to another body, we can have two approaches to deal with it

    - Parallel **(Jacobi)**:
        - Using continuous time collision detection, we can first determine the impulses on both the collisions
        - We can then aggregate the impulse vectors on all the bodies parallelly (independent of each other)
        
    - Sequential **(Gauss-Seidel)**:
        - We first detect collision between first two bodies, apply the impulse and then detect the collision with another one with new velocity of the third body

![](https://i.ibb.co/NFNccbW/Screenshot-from-2019-04-11-16-31-45.png)




- Comparison
    - A parallel approach can preserve the symmetry of the moving bodies, while sequential may break it a little
    - If we use the sequential approach, we have a sequence of Energy Conservation computation which eventually preserve the energy, but this might not be the case with parallel approach (as it might not conserve energy at the end)

## 5.10 Freezing or Re-evaluating the contact normals?

- In the above algorithm, the major problem is the termination. The following are the two problems that we might encounter in the two steps
- In the above three steps, we can process it in two ways:
    - 1st: Iterate over the three steps again and again (does not guarantee a "done" state)
    - 2nd: Freeze the 1st two steps running once and iterate over the last step again and again (all collisions may not be traced in the 1st iteration of the first two steps)

## 5.11 Inelastic Collapse

- When dealing with inelastic bodies, i.e. when COR < 1 or particularly when COR = 0, the above algorithm/approach does not converge at all
- In this condition, we might consider the velocities of the bodies to a certain precision
- Another approach is to use the above algorithm with COR=1, and then use a completely different algorithm that accounts for the actual COR of the bodies (which we will discuss in the next weeks -- **Linear Complemetarity Formulation**)
![](https://i.ibb.co/4psBqhm/Screenshot-from-2019-04-11-16-57-05.png)

## 5.12 Failsafes for Iterative Collision Response

- You do not want to iterate over and over again to detect and prevent collision in a scene
- So, we must follow a failsafe strategy

- Strategy One:
    - Just zero out all the velocities after a threshold number of iterations
    - But it is highly un-physical

## 5.13 Rigid motion failsafe

- Instead of just zeroing out all the velocities, we merge colliding bodies into one (with COR=0) and iterate it until there is no collision

![](https://i.ibb.co/HrKV4RS/Screenshot-from-2019-04-11-17-22-58.png)

## 5.14 Rigid Impact Zones

- When merging colliding bodies to a single non-colliding body, it forms a rigid impact zone